<a href="https://colab.research.google.com/github/MicaelaRomeroC3/COPs/blob/main/01_Clean_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
from datetime import datetime

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
### Establecemos que lea de esta carpeta el archivo más reciente sobre el cual aplicará la limpieza de datos

ruta_raw = '/content/drive/MyDrive/COPS/DATA/1.RAW'
ruta_clean = '/content/drive/MyDrive/COPS/DATA/2.CLEANED'

# Filtra solo archivos .xlsx
archivos = [f for f in os.listdir(ruta_raw) if f.endswith('.xlsx')]

if not archivos:
    print("⚠️ No se encontraron archivos .xlsx en la carpeta RAW.")
else:
    # Obtiene la ruta del archivo más reciente
    archivo_mas_reciente = max(
        archivos,
        key=lambda f: os.path.getmtime(os.path.join(ruta_raw, f))
    )
    ruta_archivo = os.path.join(ruta_raw, archivo_mas_reciente)

    print(f"📄 Archivo más reciente encontrado: {archivo_mas_reciente}")

    # Carga el archivo
    df = pd.read_excel(ruta_archivo)


📄 Archivo más reciente encontrado: 1 al 30 de abril final.xlsx


In [4]:
print(df.columns.tolist())

['ID', 'IDEVENTCLIENTE', 'EQUIPO', 'CIP', 'ID SERVICIO', 'SERVICIO', 'EMPRESA', 'DISPOSITIVO', 'MEDIDOR', 'FECHA EVALUACIÓN', 'HORAEVALUACION', 'VALORACIÓN', 'IDOPERADOR', 'AGENTE', 'IDEVALUADOR', 'EVALUADOR', 'IDESPECIALISTA', 'RESPUESTA', 'DURACIÓN', 'FECHA', 'HORA', 'MOTIVO', 'ALCANCE', 'CATEGORIA1', 'CATEGORIA2', 'CATEGORIA3', 'CATEGORIA4', 'GRABACIÓN', 'ARCHIVO_ADJUNTO', 'ARCHIVO_ADJUNTO2', 'ARCHIVO_ADJUNTO3', 'ARCHIVO_ADJUNTO4', 'ARCHIVO_ADJUNTO5', 'ACUERDO DE PAGO', 'PRESENTACIÓN, SALUDO CORPORATIVO Y GRABACIÓN DE LA LLAMADA', 'PRESENTACIÓN, SALUDO CORPORATIVO Y GRABACIÓN DE LA LLAMADA.COND.NO SE PRESENTA "EN NOMBRE DE LA ENTIDAD" DE LA DEUDA (MONOPRODUCTO)', 'PRESENTACIÓN, SALUDO CORPORATIVO Y GRABACIÓN DE LA LLAMADA.COND.NO SE PRESENTA "EN NOMBRE DE CAIXABANK" Y ESPECIFICA LA ENTIDAD DE CADA DEUDA (MULTIPRODUCTOS)', 'PRESENTACIÓN, SALUDO CORPORATIVO Y GRABACIÓN DE LA LLAMADA.COND.NO DICE NOMBRE AGENCIA', 'PRESENTACIÓN, SALUDO CORPORATIVO Y GRABACIÓN DE LA LLAMADA.COND.NO DICE 

In [5]:
df['FECHA EVALUACIÓN'] = pd.to_datetime(df['FECHA EVALUACIÓN'], dayfirst=True, errors='coerce')
df['AÑO_EVALUACIÓN'] = df['FECHA EVALUACIÓN'].dt.year
df['FECHA_INFORME'] = df['FECHA EVALUACIÓN'] + pd.Timedelta(days=1)
df['MES_INFORME'] = df['FECHA_INFORME'].dt.month
nombres_meses = {
    1: 'enero', 2: 'febrero', 3: 'marzo', 4: 'abril',
    5: 'mayo', 6: 'junio', 7: 'julio', 8: 'agosto',
    9: 'septiembre', 10: 'octubre', 11: 'noviembre', 12: 'diciembre'
}
df['MES_INFORME'] = df['MES_INFORME'].map(nombres_meses)
df['AÑO_INFORME'] = df['FECHA_INFORME'].dt.year
df['FECHA EVALUACIÓN'] = pd.to_datetime(df["FECHA EVALUACIÓN"]).dt.strftime("%d/%m/%Y")
df['FECHA_INFORME'] = pd.to_datetime(df["FECHA_INFORME"]).dt.strftime("%d/%m/%Y")
df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
df['AÑO_LLAMADA'] = df['FECHA'].dt.year
df['FECHA'] = pd.to_datetime(df['FECHA']).dt.strftime("%d/%m/%Y")
df = df.rename(columns={
    'FECHA': 'FECHA_LLAMADA',
    'FECHA EVALUACIÓN': 'FECHA_EVALUACIÓN'
})

In [6]:
#df.drop(['CATEGORIA1', 'CATEGORIA2', 'CATEGORIA3', 'CATEGORIA4', 'GRABACIÓN', 'ARCHIVO_ADJUNTO', 'ARCHIVO_ADJUNTO2'], axis=1, inplace=True)
#df.drop(['ARCHIVO_ADJUNTO3', 'ARCHIVO_ADJUNTO4','ARCHIVO_ADJUNTO5', 'MOTIVO', 'ALCANCE', 'DISPOSITIVO', 'IDEVENTCLIENTE', 'EQUIPO'], axis=1, inplace=True)
df['AGENTE'] = df['AGENTE'].str.split().str[0]
df[['EMPRESA_ABREV', 'PRODUCTO', 'R_NR', 'PERIMETRO', 'SEGMENTO']] = df['SERVICIO'].str.split('_', expand=True)
df['EMPRESA'] = df['EMPRESA'].replace('DIAGONAL', 'COLECTA')
df['EMPRESA'] = df['EMPRESA'].replace('GESCOBRO', 'GCBE')
df['EMPRESA_ABREV'] = df['EMPRESA_ABREV'].replace('GES', 'GCBE')
df['EMPRESA_ABREV'] = df['EMPRESA_ABREV'].replace('DIA', 'COL')
#df.drop(['OBSERVACIONES','MEDIDOR', 'IDESPECIALISTA', 'RESPUESTA', 'CIP', 'IDENTIFICACIÓN DEL INTERLOCUTOR - LOPD.COND.TITULAR', 'IDENTIFICACIÓN DEL INTERLOCUTOR - LOPD.COND.AVALISTA'],axis=1, inplace=True)
df = df.rename(columns={'PRESENTACIÓN, SALUDO CORPORATIVO Y GRABACIÓN DE LA LLAMADA': 'P1'})
df = df.rename(columns={'PRESENTACIÓN, SALUDO CORPORATIVO Y GRABACIÓN DE LA LLAMADA.COND.NO SE PRESENTA "EN NOMBRE DE LA ENTIDAD" DE LA DEUDA (MONOPRODUCTO)': 'P11'})
df = df.rename(columns={'PRESENTACIÓN, SALUDO CORPORATIVO Y GRABACIÓN DE LA LLAMADA.COND.NO DICE NOMBRE+APELLIDO GESTOR': 'P12'})
df = df.rename(columns={'PRESENTACIÓN, SALUDO CORPORATIVO Y GRABACIÓN DE LA LLAMADA.COND.NO DICE NOMBRE AGENCIA': 'P13'})
df = df.rename(columns={'PRESENTACIÓN, SALUDO CORPORATIVO Y GRABACIÓN DE LA LLAMADA.COND.NO INFORMA DE LA GRABACIÓN DE LA LLAMADA': 'P14'})
df = df.rename(columns={'PRESENTACIÓN, SALUDO CORPORATIVO Y GRABACIÓN DE LA LLAMADA.COND.NO SE PRESENTA "EN NOMBRE DE CAIXABANK" Y ESPECIFICA LA ENTIDAD DE CADA DEUDA (MULTIPRODUCTOS)': 'P15'})
df = df.rename(columns={'IDENTIFICACIÓN DEL INTERLOCUTOR - LOPD': 'P2'})
df = df.rename(columns={'IDENTIFICACIÓN DEL INTERLOCUTOR - LOPD.COND.NO VERIFICA DNI': 'P21'})
df = df.rename(columns={'IDENTIFICACIÓN DEL INTERLOCUTOR - LOPD.COND.NO PREGUNTA POR EL CLIENTE DEUDOR': 'P22'})
df = df.rename(columns={'IDENTIFICACIÓN DEL INTERLOCUTOR - LOPD.COND.INCUMPLIMIENTO COMPLIANCE GRABACIÓN LLAMADAS (NUMERACIÓN TARJETA)': 'P23'})
df = df.rename(columns={'ESCUCHA COMPRENSIVA Y EMPÁTICA': 'P3'})
df = df.rename(columns={'ESCUCHA COMPRENSIVA Y EMPÁTICA.COND.NO DEMUESTRA INTERÉS POR LA SITUACIÓN DEL CLIENTE': 'P31'})
df = df.rename(columns={'ESCUCHA COMPRENSIVA Y EMPÁTICA.COND.NO TIENEN EN CUENTA LA INFORMACIÓN TRASLADADA POR EL CLIENTE PARA CONSEGUIR UN ACUERDO': 'P32'})
df = df.rename(columns={'ESCUCHA COMPRENSIVA Y EMPÁTICA.COND.ADOPTA UNA ACTITUD INAPROPIADA DE CARA AL CLIENTE (SARCÁSTICA, CONDESCENDIENTE, DE REPROCHE, ...)': 'P33'})
df = df.rename(columns={'TRATO CORDIAL AL CLIENTE': 'P4'})
df = df.rename(columns={'DESPEDIDA CORPORATIVA': 'P5'})
df = df.rename(columns={'RESPONSABILIDAD  CORPORATIVA': 'P6'})
df = df.rename(columns={'RESPONSABILIDAD  CORPORATIVA.COND.NO SE DISCULPA ANTE LAS MOLESTIAS EVIDENCIADAS POR EL CLIENTE DURANTE LA LLAMADA': 'P61'})
df = df.rename(columns={'RESPONSABILIDAD  CORPORATIVA.COND.LA FALTA DE PROFESIONALIDAD DEL GESTOR  TRASLADA UNA IMAGEN NEGATIVA DE CAIXABANK': 'P62'})
columnas_a_modificar = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6']
for col in columnas_a_modificar:
    df[col] = df[col].map({'Sí': True, 'No': False})
df = df.rename(columns={'INFORMACIÓN MOTIVO DE LA LLAMADA': 'S1'})
df = df.rename(columns={'INFORMACIÓN MOTIVO DE LA LLAMADA.COND.NO ESPECIFICA CLARAMENTE EL MOTIVO DE LA LLAMADA': 'S11'})
df = df.rename(columns={'INFORMACIÓN MOTIVO DE LA LLAMADA.COND.NO ESPECIFICA EL IMPORTE DE LA DEUDA': 'S12'})
df = df.rename(columns={'INFORMACIÓN MOTIVO DE LA LLAMADA.COND.NO ESPECIFICA LA ANTIGÜEDAD DE LA DEUDA': 'S13'})
df = df.rename(columns={'SITUACIÓN ACTUAL DEL DEUDOR': 'S2'})
df = df.rename(columns={'PREPARACIÓN CIERRE : RESUMEN DE ACUERDOS': 'S5'})
df = df.rename(columns={'PREPARACIÓN CIERRE : RESUMEN DE ACUERDOS.COND.NO CONCRETA FECHA E IMPORTE EN CASO DE ACUERDO DE PAGO': 'S51'})
df = df.rename(columns={'PREPARACIÓN CIERRE : RESUMEN DE ACUERDOS.COND.NO CONCRETA FECHA DE SEGUIMIENTO EN CASO DE NO HABER ACUERDO DE PAGO': 'S52'})
df = df.rename(columns={'SONDEO: IDENTIFICAR CAPACIDAD DE PAGO Y DETECCIÓN DE NECESIDADES': 'S3'})
df = df.rename(columns={'SONDEO: IDENTIFICAR CAPACIDAD DE PAGO Y DETECCIÓN DE NECESIDADES.COND.NO SE IDENTIFICA LA CAPACIDAD DE PAGO': 'S31'})
df = df.rename(columns={'SONDEO: IDENTIFICAR CAPACIDAD DE PAGO Y DETECCIÓN DE NECESIDADES.COND.EL GESTOR NO HACE REFERENCIA AL SIGUIENTE VENCIMIENTO CUANDO CIERRAN UN ACUERDO':'S32'})
df = df.rename(columns={'SONDEO: IDENTIFICAR CAPACIDAD DE PAGO Y DETECCIÓN DE NECESIDADES.COND.NO SONDEA, PRESUPONE LA SITUACIÓN DEL CLIENTE': 'S33'})
df = df.rename(columns={'SONDEO: IDENTIFICAR CAPACIDAD DE PAGO Y DETECCIÓN DE NECESIDADES.COND.INTENTA RESOLVER LA LLAMADA RÁPIDO (SE QUITA LA LLAMADA DE ENCIMA)': 'S34'})
df = df.rename(columns={'DIRECCIÓN': 'S4'})
df = df.rename(columns={'DIRECCIÓN.COND.PASIVO: NO SE PERCIBE QUE EL GESTOR DOMINE LA CONVERSACIÓN, MÁS BIEN ÉSTA ES DIRIGIDA POR EL CLIENTE': 'S41'})
df = df.rename(columns={'DIRECCIÓN.COND.DESESTRUCTURADO: NO CONSIGUE LINEALIDAD Y CONCRECIÓN': 'S42'})
df = df.rename(columns={'DIRECCIÓN.COND.REACTIVO: ACTÚA “A LA DEFENSIVA”': 'S43'})
df = df.rename(columns={'DIRECCIÓN.COND.PERJUDICIAL: ANTE UN CLIENTE COMPLEJO/INSATISFECHO GENERA CONFRONTACIÓN': 'S44'})
df = df.rename(columns={'DIRECCIÓN.COND.ALARGA O PRECIPITA DE MANERA INNECESARIA EL CIERRE DE CONVERSACIÓN': 'S45'})
df = df.rename(columns={'DIRECCIÓN.COND.NO DIRIGE LA SECUENCIA DE LLAMADA MANTENIENDO EL ENFOQUE': 'S46'})
columnas_a_modificarS = ['S1', 'S2', 'S3', 'S4', 'S5']

for col in columnas_a_modificarS:
    df[col] = df[col].apply(lambda x: True if x == 'Sí' else False if x == 'No' else 'No Procede')

df = df.rename(columns={'USO EFECTIVO DEL SCRIPT': 'T1'})
df = df.rename(columns={'USO EFECTIVO DEL SCRIPT.COND.NO HACE USO EFECTIVO DEL SCRIPT (TODOS LOS CASOS SALVO PROTOCOLO NEG. OFICINA)': 'T11'})
df = df.rename(columns={'USO EFECTIVO DEL SCRIPT.COND.NEG OFI_1: NO SIGUE PROCEDIMIENTO NEGOCIACIÓN OFICINA COMO OBJECIÓN CLIENTE': 'T12'})
df = df.rename(columns={'USO EFECTIVO DEL SCRIPT.COND.NEG OFI_2: INFORMACIÓN INSUFICIENTE DEL CLIENTE_ NO SIGUE PROCEDIMIENTO': 'T13'})
df = df.rename(columns={'USO EFECTIVO DEL SCRIPT.COND.NEG OFI_3: INFORMACIÓN SUFICIENTE DEL CLIENTE_ NO SIGUE PROCEDIMIENTO': 'T14'})
df = df.rename(columns={'BUSCAR EL COMPROMISO DE PAGO ENUNCIANDO LOS BENEFICIOS Y LAS CONSECUENCIAS DE LA REGULARIZACIÓN': 'T2'})
df = df.rename(columns={'BUSCAR EL COMPROMISO DE PAGO ENUNCIANDO LOS BENEFICIOS Y LAS CONSECUENCIAS DE LA REGULARIZACIÓN.COND.NO BUSCA EL COMPROMISO DE PAGO ENUNCIANDO BENEFICIOS/CONSECUENCIAS': 'T21'})
df = df.rename(columns={'BUSCAR EL COMPROMISO DE PAGO ENUNCIANDO LOS BENEFICIOS Y LAS CONSECUENCIAS DE LA REGULARIZACIÓN.COND.NO ES EXIGENTE ACORDANDO UNA FECHA': 'T22'})
df = df.rename(columns={'CAPACIDAD  DE EXPRESIÓN/ NEGOCIACIÓN': 'T3'})
df = df.rename(columns={'CAPACIDAD  DE EXPRESIÓN/ NEGOCIACIÓN.COND.NO ADECÚA SU LENGUAJE Y SU FORMA DE EXPRESARSE AL GRADO DE COMPRESIÓN DEL CLIENTE': 'T31'})
df = df.rename(columns={'CAPACIDAD  DE EXPRESIÓN/ NEGOCIACIÓN.COND.NO SE EXPRESA CON CLARIDAD Y PROFESIONALIDAD': 'T32'})
df = df.rename(columns={'CAPACIDAD  DE EXPRESIÓN/ NEGOCIACIÓN.COND.NO INTERACTÚA EN LA COMUNICACIÓN CON EL CLIENTE': 'T33'})
df = df.rename(columns={'CAPACIDAD  DE EXPRESIÓN/ NEGOCIACIÓN.COND.UTILIZA UN TONO DE VOZ INAPROPIADO (MECÁNICO/ AGRESIVO/PASIVO...)': 'T34'})
df = df.rename(columns={'CAPACIDAD  DE EXPRESIÓN/ NEGOCIACIÓN.COND.NO MUESTRA SEGURIDAD EN LA COMUNICACIÓN': 'T35'})
df = df.rename(columns={'CAPACIDAD  DE EXPRESIÓN/ NEGOCIACIÓN.COND.NO AFRONTA LAS OBJECIONES CON ARGUMENTOS ADECUADOS, CORRECTOS Y COMPLETOS': 'T36'})
df = df.rename(columns={'CAPACIDAD  DE EXPRESIÓN/ NEGOCIACIÓN.COND.NO APLICA UNA SOLUCIÓN ADAPTADA AL PERFIL DEL CLIENTE': 'T37'})
columnas_a_modificarT = ['T1', 'T2', 'T3']
for col in columnas_a_modificarT:
    df[col] = df[col].apply(lambda x: True if x == 'Sí' else False if x == 'No' else 'No Procede')
df = df.rename(columns={'OFRECER TPV COMO ALTERNATIVA PRIORITARIA DE PAGO': 'C1'})
df = df.rename(columns={'OFRECER TPV COMO ALTERNATIVA PRIORITARIA DE PAGO.COND.NO OFRECE TARJETA DE DÉBITO CAIXA': 'C11'})
df = df.rename(columns={'OFRECER TPV COMO ALTERNATIVA PRIORITARIA DE PAGO.COND.NO OFRECE TARJETA DE CRÉDITO OTRA ENTIDAD': 'C12'})
df = df.rename(columns={'OFRECER TPV COMO ALTERNATIVA PRIORITARIA DE PAGO.COND.NO OFRECE TARJETA DE FAMILIAR/AMIGO': 'C13'})
df = df.rename(columns={'INFORMAR DE LAS VENTAJAS DEL PAGO POR TPV': 'C14'})
df = df.rename(columns={'INFORMAR DE LAS VENTAJAS DEL PAGO POR TPV.COND.NO INFORMA QUE LA FORMA DE PAGO 100% SEGURA': 'C15'})
df = df.rename(columns={'INFORMAR DE LAS VENTAJAS DEL PAGO POR TPV.COND.NO INFORMA DE LA COMODIDAD E INMEDIATEZ DEL PAGO': 'C16'})
df = df.rename(columns={'INFORMAR DE LAS VENTAJAS DEL PAGO POR TPV.COND.NO INFORMA QUE SUPONE MAYOR AHORRO': 'C17'})
df = df.rename(columns={'INFORMAR DE LAS VENTAJAS DEL PAGO POR TPV.COND.NO INFORMA QUE ES ADAPTABLE SEGÚN SUS NECESIDADES': 'C18'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR TPV': 'C19'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR TPV.COND.TARJETA DE DÉBITO CAIXA': 'C110'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR TPV.COND.TARJETA DE CRÉDITO OTRA ENTIDAD': 'C111'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR TPV.COND.TARJETA DE FAMILIAR/AMIGO': 'C112'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR TPV.COND.NO SE ESPECIFICA TIPO DE TARJETA': 'C113'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR TPV.COND.NO DISPONE DE TARJETA DE DÉBITO EN CAIXA': 'C114'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR TPV.COND.NO DISPONE DE TARJETA DE CRÉDITO EN OTRA ENTIDAD': 'C115'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR TPV.COND.NO DISPONE DE NINGUNA TARJETA': 'C116'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR TPV.COND.NO TENGO NINGÚN FAMILIAR/AMIGO QUE ME PUEDA FACILITAR UNA TARJETA': 'C117'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR TPV.COND.NO PUEDE CONSEGUIR UNA TARJETA DE UN TERCERO EN EL MOMENTO DE LA LLAMADA': 'C118'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR TPV.COND.TARJETA BLOQUEADA': 'C119'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR TPV.COND.DESCONOCE LA OPERATIVA DE PAGO CON TARJETA POR TELÉFONO Y QUE LE PROVOCA DESCONFIANZA': 'C120'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR TPV.COND.NO DISPONE DE DINERO PARA SALDAR LA DEUDA': 'C121'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR TPV.COND.OTROS': 'C122'})
df = df.rename(columns={'CUMPLIMIENTO PROTOCOLO ACEPTACIÓN TPV': 'C123'})
df = df.rename(columns={'PRIORIZA LOS CANALES DIGITALES U OTROS MEDIOS PARA REALIZAR LOS INGRESOS LO ANTES POSIBLE': 'C2'})
df = df.rename(columns={'PRIORIZA LOS CANALES DIGITALES U OTROS MEDIOS PARA REALIZAR LOS INGRESOS LO ANTES POSIBLE.COND.NO OFRECEN LAS ALTERNATIVAS DE PAGO DISPONIBLES (APARTE DE TPV)': 'C21'})
df = df.rename(columns={'PRIORIZA LOS CANALES DIGITALES U OTROS MEDIOS PARA REALIZAR LOS INGRESOS LO ANTES POSIBLE.COND.ENFOQUE INCORRECTO CON EXCESO DE ACUERDOS PARCIALES': 'C22'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR OTRO CANAL (NO TPV)': 'C23'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR OTRO CANAL (NO TPV).COND.PAGO A TRAVÉS DE UNA TRANSFERENCIA VÍA WEB': 'C24'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR OTRO CANAL (NO TPV).COND.REALIZANDO UN INGRESO EN CAJERO': 'C25'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR OTRO CANAL (NO TPV).COND.REALIZANDO UN INGRESO EN OFICINA': 'C26'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR OTRO CANAL (NO TPV).COND.PAGO A TERCEROS_APP CAIXA (EN RENTING, CHM, COMERCIA)': 'C27'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR OTRO CANAL (NO TPV).COND.ACUERDO PARCIAL OFRECIDO POR EL GESTOR (ULTIMA ALTERNATIVA O PROPUESTA DEL CLIENTE)': 'C28'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR OTRO CANAL (NO TPV).COND.ACUERDO PARCIAL OFRECIDO POR EL GESTOR (ALTERNATIVA PRIORITARIA)': 'C29'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR OTRO CANAL (NO TPV).COND.OTRA OPCIÓN': 'C211'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR OTRO CANAL (NO TPV).COND.CONFINAMIENTO Y LA OFICINA O CAJERO ESTÁ FUERA DE LA ZONA CONFINADA': 'C212'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR OTRO CANAL (NO TPV).COND.HOSPITALIZACIÓN': 'C213'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR OTRO CANAL (NO TPV).COND.MIEDO POR CONTAGIO DE COVID': 'C214'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR OTRO CANAL (NO TPV).COND.LA RELACIÓN CON LOS EMPLEADOS DE LA SUCURSAL NO ES BUENA': 'C215'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR OTRO CANAL (NO TPV).COND.NO DISPONE DE DINERO PARA SALDAR LA DEUDA': 'C216'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR OTRO CANAL (NO TPV).COND.SIN ACCESO A INTERNET': 'C217'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR OTRO CANAL (NO TPV).COND.FALTA MANEJO DE LA WEB O APLICACIONES DE CAIXA': 'C218'})
df = df.rename(columns={'ACEPTACIÓN ACUERDO DE PAGO POR OTRO CANAL (NO TPV).COND.OTROS': 'C219'})
columnas_a_modificarC = ['C1', 'C14', 'C19', 'C2', 'C123', 'C23']
for col in columnas_a_modificarC:
    df[col] = df[col].apply(lambda x: True if x == 'Sí' else False if x == 'No' else 'No Procede')
df = df.rename(columns={'ID SERVICIO': 'ID_SERVICIO'})
df = df.rename(columns={'ACUERDO DE PAGO': 'ACUERDO_DE_PAGO'})
df = df.rename(columns={'EVITA EL RIESGO REPUTACIONAL PARA CAIXABANK DURANTE TODA LA CONVERSACIÓN': 'E1'})
df = df.rename(columns={'EVITA EL RIESGO REPUTACIONAL PARA CAIXABANK DURANTE TODA LA CONVERSACIÓN.COND.DEUDORES HIPOTECARIOS EN RIESGO DE EXCLUSIÓN  Y RIESGO REPUTACIONAL': 'E11'})
df = df.rename(columns={'EVITA EL RIESGO REPUTACIONAL PARA CAIXABANK DURANTE TODA LA CONVERSACIÓN.COND.QUEJAS / RECLAMACIONES / AMENAZAS': 'E12'})
df = df.rename(columns={'¿LA ACCIÓN DEL GESTOR DURANTE LA LLAMADA CONSIGUE MODIFICAR LA ACTITUD INICIAL DEL CLIENTE?':'A1'})
df=  df.rename(columns={'¿LA ACCIÓN DEL GESTOR DURANTE LA LLAMADA CONSIGUE MODIFICAR LA ACTITUD INICIAL DEL CLIENTE?.COND.INCREMENTA LA PERCEPCIÓN NEGATIVA DEL CLIENTE': 'A11'})
df = df.rename(columns={'¿LA ACCIÓN DEL GESTOR DURANTE LA LLAMADA CONSIGUE MODIFICAR LA ACTITUD INICIAL DEL CLIENTE?.COND.REDUCE LA PERCEPCIÓN NEGATIVA DEL CLIENTE': 'A12'})
df = df.rename(columns={'¿LA ACCIÓN DEL GESTOR DURANTE LA LLAMADA CONSIGUE MODIFICAR LA ACTITUD INICIAL DEL CLIENTE?.COND.INCREMENTA LA PERCEPCIÓN POSITIVA DEL CLIENTE': 'A13'})
df = df.rename(columns={'¿LA ACCIÓN DEL GESTOR DURANTE LA LLAMADA CONSIGUE MODIFICAR LA ACTITUD INICIAL DEL CLIENTE?.COND.LA CAMBIA DE SATISFECHO A INSATISFECHO':'A14'})
columnas_a_modificarF = ['E1','A1']
for col in columnas_a_modificarF:
    df[col] = df[col].map({'Sí': True, 'No': False})
df['S5M'] = df['S5'].apply(
    lambda x: 1 if x == 1 or x == 'No Procede' else 0 if x == 0 else pd.NA)
df['C1M'] = df['C1'].apply(
    lambda x: 1 if x == 1 or x == 'No Procede' else 0 if x == 0 else pd.NA)
df['C14M'] = df['C14'].apply(
    lambda x: 1 if x == 1 or x == 'No Procede' else 0 if x == 0 else pd.NA)
df['C19M'] = df['C19'].apply(
    lambda x: 1 if x == 1 or x == 'No Procede' else 0 if x == 0 else pd.NA)
df['C123M'] = df['C123'].apply(
    lambda x: 1 if x == 1 or x == 'No Procede' else 0 if x == 0 else pd.NA)
df['C2M'] = df['C2'].apply(
    lambda x: 1 if x == 1 or x == 'No Procede' else 0 if x == 0 else pd.NA)
df['C23M'] = df['C23'].apply(
    lambda x: 1 if x == 1 or x == 'No Procede' else 0 if x == 0 else pd.NA)
df['T1M'] = df['T1'].apply(
    lambda x: 1 if x == 1 or x == 'No Procede' else 0 if x == 0 else pd.NA)
df['T2M'] = df['T2'].apply(
    lambda x: 1 if x == 1 or x == 'No Procede' else 0 if x == 0 else pd.NA)
cols_booleanas = ['S5M', 'C1M', 'T1M', 'T2M']
df[cols_booleanas] = df[cols_booleanas].astype(bool)

In [7]:
import unicodedata

# 🧾 Lista de columnas a eliminar (sin acentos ni caracteres especiales)
cols_a_eliminar = [
    'CATEGORIA1', 'CATEGORIA2', 'CATEGORIA3', 'CATEGORIA4',
    'GRABACIÓN', 'ARCHIVO_ADJUNTO', 'ARCHIVO_ADJUNTO2',
    'ARCHIVO_ADJUNTO3', 'ARCHIVO_ADJUNTO4', 'ARCHIVO_ADJUNTO5',
    'MOTIVO', 'ALCANCE', 'DISPOSITIVO', 'IDEVENTCLIENTE',
    'EQUIPO', 'OBSERVACIONES', 'MEDIDOR', 'IDESPECIALISTA',
    'RESPUESTA', 'CIP',
    'IDENTIFICACIÓN DEL INTERLOCUTOR - LOPD.COND.TITULAR',
    'IDENTIFICACION DEL INTERLOCUTOR - LOPD.COND.AVALISTA'
]

# 🧠 Función para normalizar texto (quita tildes, mayúsculas, espacios)
def normalizar(txt):
    txt = str(txt).strip().upper()
    txt = ''.join(ch for ch in unicodedata.normalize('NFD', txt)
                  if unicodedata.category(ch) != 'Mn')
    return txt

# Normalizar nombres de columnas actuales
cols_norm = {normalizar(c): c for c in df.columns}

# Determinar qué columnas conservar (todas menos las eliminadas)
cols_a_eliminar_norm = [normalizar(c) for c in cols_a_eliminar]
cols_a_conservar = [
    orig for norm, orig in cols_norm.items()
    if norm not in cols_a_eliminar_norm
]

# Crear nuevo DataFrame limpio
df_limpio = df[cols_a_conservar].copy()

print(f"✅ Columnas originales: {len(df.columns)}")
print(f"🧹 Columnas eliminadas: {len(df.columns) - len(df_limpio.columns)}")
print(f"📊 Columnas finales: {len(df_limpio.columns)}")


✅ Columnas originales: 162
🧹 Columnas eliminadas: 22
📊 Columnas finales: 140


In [8]:
fecha_referencia = datetime.today().strftime('%Y%m%d')
nombre_base = f"llamadas_clean_{fecha_referencia}"
nombre_salida = f"{nombre_base}.csv"
ruta_salida = os.path.join(ruta_clean, nombre_salida)
### Pondremos que evitar sobrescribir si ya existe
contador = 1
while os.path.exists(ruta_salida):
    ruta_salida = os.path.join(ruta_clean, f"{nombre_base}_{contador}.csv")
    contador += 1

df_limpio.to_csv(ruta_salida, index=False, encoding='utf-8-sig')
print(f"\n✅ Archivo limpio guardado en: {ruta_salida}")


✅ Archivo limpio guardado en: /content/drive/MyDrive/COPS/DATA/2.CLEANED/llamadas_clean_20251024.csv
